# Fetch and inspect well data

In [ ]:
import os

# Set datapath properly
path = os.path.abspath('../data/interim/DATAPATH')
path = path + os.path.sep
path

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import numpy as np

import rsf.api as sf

# Set a custom DATAPATH to Madagascar
%env DATAPATH={path}

In [ ]:
sns.set_style('ticks')

## Well metadata table

| Original name | Alias | Keyword | Value |
|---------------|-------|---------|-------|
| Well 4 | Well A | Kelly bushing | 17.5 m |
| Well 4 | Well A | Water depth | 355 m |
| Well 4 | Well A | Shotpoint | 440 |
| Well 4 | Well A | CDP | 808 |
| Well 5 | Well B | Kelly bushing | 26.5 m |
| Well 5 | Well B | Water depth | 355.5 m |
| Well 5 | Well B | Shotpoint | 822 |
| Well 5 | Well B | CDP | 1572|
| Well 7 | Well C | Kelly bushing | 17.5 m |
| Well 7 | Well C | Shotpoint | 310 |
| Well 7 | Well C | CDP | 548 |

In [ ]:
metadata = {
    'Well 4' : {
        'ALIAS' : 'Well A',
        'KB' : 17.5,
        'WD' : 355,
        'EP' : 440,
        'CDP' : 808
    },
    
    'Well 5' : {
        'ALIAS' : 'Well B',
        'KB' : 26.5,
        'WD' : 355.5,
        'EP' : 822,
        'CDP' : 1572
    },
    
    'Well 7' : {
        'ALIAS' : 'Well C',
        'KB' : 17.5, # not 100% sure about this
        'EP' : 310,
        'CDP' : 548
    }
}

In [ ]:
# Fetching the data from the web
!wget http://s3.amazonaws.com/open.source.geoscience/open_data/Mobil_Avo_Viking_Graben_Line_12/mobil_wellogs.tar.gz

In [ ]:
# Now let's check the data integrity
# The MD5 checksum should be equal to 5cfbbabef17d96ea5bc0ef791ad62afb

!md5sum mobil_wellogs.tar.gz

In [ ]:
# Now, lets extract the data
!tar xvzf mobil_wellogs.tar.gz

# Analysing Well 4

In [ ]:
# First, lets inspect the .blk file
!head well4.blk | cat -n

In [ ]:
# Now the .elan file
!head -15 well4.elan | cat -n

In [ ]:
# Now the .log
!head -15 well4.log | cat -n

In [ ]:
# Finaly the .Q file
!head well4.Q | cat -n

All well files are logs indexed by measured depth, no information on kelly bushing or any other datum elevation. It's easy enough to read these files using pandas, but we may need to resample if we want to write all logs to a single LAS file. Also, all log units are on S.I., just as God intended it.

In [ ]:
logs = pd.read_csv('well4.log', skiprows=14,
                    names = ['DEPT', 'GR', 'PR', 'RHOB', 'CALI', 'VELS', 'VELC'],
                  sep='\s+', na_values=-999.25)

logs.head()

In [ ]:
# Lets check if this file has constant sampling depth
logs.DEPT.diff().unique()

In [ ]:
vol = pd.read_csv('well4.elan', skiprows=14,
                      names = ['DEPT', 'VILL', 'VXBW', 'VCOA', 'VQUA',
                             'VCLC', 'VPAR', 'VXOI', 'VXGA', 'VXWA', 'SDR'],
                  sep='\s+', na_values=-999.25)
cols = list(vol)
cols.remove('DEPT')

vol.loc[vol.SDR == 999.25, cols] = np.nan


vol.head()

In [ ]:
# Now the .elan file
!head -14 well4.elan | cat -n

In [ ]:
cols

In [ ]:
# Facecolors for the volume plot
facecolor = {
    'VILL' : sns.xkcd_rgb['forest'],
    'VXBW' : sns.xkcd_rgb['royal blue'],
    'VQUA' : sns.xkcd_rgb['yellow'],
    'VCLC' : sns.xkcd_rgb['blue green'],
    'VPAR' : sns.xkcd_rgb['grey'],
    'VCOA' : sns.xkcd_rgb['black'],
    
    'VXWA' : sns.xkcd_rgb['aqua'],
    'VXOI' : sns.xkcd_rgb['bright green'],
    'VXGA' : sns.xkcd_rgb['bright red']
}

hatch = {
    'VILL' : '-',
    'VXBW' : '-',
    'VQUA' : '.',
    'VCLC' : '+',
    'VPAR' : None,
    'VCOA' : None,
    
    'VXWA' : None,
    'VXOI' : None,
    'VXGA' : None
}

In [ ]:
# Checking volume sums

cols = list(vol)
cols.remove('DEPT')
cols.remove('SDR')
plt.plot(vol[cols].sum(axis=1), '.')

In [ ]:
from matplotlib.ticker import AutoMinorLocator
minor_locator = AutoMinorLocator(10)

In [ ]:
plt.figure(figsize=(10,100))
ax = plt.subplot(151)

# Caliper
plt.plot(logs.CALI, logs.DEPT, 'k', label='CALI', lw=1)
plt.xlim(200, 600)
plt.gca().xaxis.tick_top()
plt.gca().set_xlabel('CALI (mm)')    
plt.gca().xaxis.set_label_position('top') 
plt.locator_params(axis='x', nbins=3)
plt.yticks(np.arange(np.floor(logs.DEPT.min() / 100) * 100,
                     np.ceil(logs.DEPT.max()/100) * 100 + 100, 100),
          rotation=90)

plt.gca().yaxis.set_minor_locator(minor_locator)
plt.grid()

# Gamma Ray
plt.subplot(152, sharey=ax)

## Plot normal curve
cutoff = np.percentile(logs[logs.GR.notnull()].GR, 50)

plt.fill_betweenx(logs.DEPT, logs.GR, cutoff, where= logs.GR > cutoff,
                  facecolor=sns.xkcd_rgb['puke green'])
plt.fill_betweenx(logs.DEPT, logs.GR, cutoff, where= logs.GR < cutoff,
                  facecolor=sns.xkcd_rgb['yellow'])

plt.plot(logs.GR, logs.DEPT, 'k-', label='GR', lw=1)

## Plot backup
plt.plot(logs.GR - 150, logs.DEPT, 'k--', label='GR', lw=1)

plt.xlim(0, 150)


plt.gca().xaxis.tick_top()
plt.gca().set_xlabel('GR (GAPI)')    
plt.gca().xaxis.set_label_position('top') 

plt.setp(plt.gca().get_yticklabels(), visible=False)
plt.locator_params(axis='x', nbins=3)
plt.grid()

# Plot density
plt.subplot(153, sharey=ax)

## Normal curve
plt.plot(logs.RHOB, logs.DEPT, color=sns.xkcd_rgb['brick'], label='RHOB', lw=1)

## Plot backup
plt.plot(logs.RHOB + 1000, logs.DEPT, ls='--', color=sns.xkcd_rgb['brick'],
         label='RHOB', lw=1)

plt.gca().xaxis.tick_top()
plt.gca().set_xlabel(u'RHOB (kg/m³)')    
plt.gca().xaxis.set_label_position('top') 

plt.setp( plt.gca().get_yticklabels(), visible=False)
plt.gca().set_xticks([1950, 2200, 2450, 2700, 2950])
plt.gca().set_xticklabels([1950, '', 2450, '', 2950])
plt.grid()

plt.xlim(2950, 1950)

# Plot velocity
plt.subplot(154, sharey=ax)

plt.plot(logs.VELC, logs.DEPT, color=sns.xkcd_rgb['blue green'], label='VELC', lw=1)
plt.plot(logs.VELS, logs.DEPT, color=sns.xkcd_rgb['blood red'], label='VELS', lw=1)


plt.gca().xaxis.tick_top()
plt.gca().set_xlabel('Velocity (km/s)')    
plt.gca().xaxis.set_label_position('top') 

plt.setp( plt.gca().get_yticklabels(), visible=False)
plt.locator_params(axis='x', nbins=3)
plt.grid()

# Plot Volumes
plt.subplot(155, sharey=ax)

y = np.zeros(len(vol))
    
for i, col in enumerate(['VILL', 'VXBW', 'VQUA', 'VCLC', 'VPAR', 'VCOA', 'VXWA', 'VXOI', 'VXGA']):
    y += vol[col]
    plt.fill_betweenx(vol.DEPT, y, 0,
                  facecolor=facecolor[col],
                 zorder=-i, lw=0, label=col,
                     hatch=hatch[col])

#plt.legend(loc='upper center', bbox_to_anchor=(-2.95, 1.07), ncol=5)
plt.gca().xaxis.tick_top()
plt.gca().set_xlabel('Volume')    
plt.gca().xaxis.set_label_position('top') 

plt.setp( plt.gca().get_yticklabels(), visible=False)
plt.locator_params(axis='x', nbins=3)
plt.grid(axis='y')
plt.xlim(0,1)

plt.autoscale(enable=True, axis='y', tight=True)
#plt.ylim(1500, 2500)
plt.ylim(plt.ylim()[::-1])

plt.tight_layout()

plt.savefig('well_4_composite_log.pdf')

plt.close()

# Analysing well 5

In [ ]:
# First, lets inspect the .blk file
!head well5.blk | cat -n

In [ ]:
# Now the .log
!head -15 well5.log | cat -n

In [ ]:
# Finaly the .Q file
!head well5.Q | cat -n

Well 5 has the same log set as Well 4. So we just repeat the drill.

In [ ]:
logs = pd.read_csv('well5.log', skiprows=14,
                    names = ['DEPT', 'GR', 'PR', 'RHOB', 'CALI', 'VELC', 'VELS'],
                  sep='\s+', na_values=[-999.25, -9999.25])

logs.head()

In [ ]:
# Lets check if this file has constant sampling depth
logs.DEPT.diff().unique()

In [ ]:
# Now the .elan file
!head -15 well5.elan | cat -n

In [ ]:
# There's to coal on well 5

vol = pd.read_csv('well5.elan', skiprows=14,
                      names = ['DEPT', 'VILL', 'VXBW', 'VCLC',
                             'VQUA', 'VPAR', 'VXOI', 'VXGA', 'VXWA', 'SDR'],
                  sep='\s+', na_values=-999.25)
cols = list(vol)
cols.remove('DEPT')

vol.loc[vol.SDR == 999.25, cols] = np.nan


vol.head()

## Ploting Well 5 composite log

In [ ]:
# Checking volume sums

cols = list(vol)
cols.remove('DEPT')
cols.remove('SDR')
plt.plot(vol[cols].sum(axis=1), '.')

In [ ]:
plt.figure(figsize=(10,100))
ax = plt.subplot(151)

# Caliper
plt.plot(logs.CALI, logs.DEPT, 'k', label='CALI', lw=1)
plt.xlim(200, 600)
plt.gca().xaxis.tick_top()
plt.gca().set_xlabel('CALI (mm)')    
plt.gca().xaxis.set_label_position('top') 
plt.locator_params(axis='x', nbins=3)
plt.yticks(np.arange(np.floor(logs.DEPT.min() / 100) * 100,
                     np.ceil(logs.DEPT.max()/100) * 100 + 100, 100),
          rotation=90)

plt.gca().yaxis.set_minor_locator(minor_locator)
plt.grid()

# Gamma Ray
plt.subplot(152, sharey=ax)

## Plot normal curve
cutoff = np.percentile(logs[logs.GR.notnull()].GR, 50)

plt.fill_betweenx(logs.DEPT, logs.GR, cutoff, where= logs.GR > cutoff,
                  facecolor=sns.xkcd_rgb['puke green'])
plt.fill_betweenx(logs.DEPT, logs.GR, cutoff, where= logs.GR < cutoff,
                  facecolor=sns.xkcd_rgb['yellow'])

plt.plot(logs.GR, logs.DEPT, 'k-', label='GR', lw=1)

## Plot backup
plt.plot(logs.GR - 150, logs.DEPT, 'k--', label='GR', lw=1)

plt.xlim(0, 150)


plt.gca().xaxis.tick_top()
plt.gca().set_xlabel('GR (GAPI)')    
plt.gca().xaxis.set_label_position('top') 

plt.setp(plt.gca().get_yticklabels(), visible=False)
plt.locator_params(axis='x', nbins=3)
plt.grid()

# Plot density
plt.subplot(153, sharey=ax)

## Normal curve
plt.plot(logs.RHOB, logs.DEPT, color=sns.xkcd_rgb['brick'], label='RHOB', lw=1)

## Plot backup
plt.plot(logs.RHOB + 1000, logs.DEPT, ls='--', color=sns.xkcd_rgb['brick'],
         label='RHOB', lw=1)

plt.gca().xaxis.tick_top()
plt.gca().set_xlabel(u'RHOB (kg/m³)')    
plt.gca().xaxis.set_label_position('top') 

plt.setp( plt.gca().get_yticklabels(), visible=False)
plt.gca().set_xticks([1950, 2200, 2450, 2700, 2950])
plt.gca().set_xticklabels([1950, '', 2450, '', 2950])
plt.grid()

plt.xlim(2950, 1950)

# Plot velocity
plt.subplot(154, sharey=ax)

plt.plot(logs.VELC, logs.DEPT, color=sns.xkcd_rgb['blue green'], label='VELC', lw=1)
plt.plot(logs.VELS, logs.DEPT, color=sns.xkcd_rgb['blood red'], label='VELS', lw=1)


plt.gca().xaxis.tick_top()
plt.gca().set_xlabel('Velocity (km/s)')    
plt.gca().xaxis.set_label_position('top') 

plt.setp( plt.gca().get_yticklabels(), visible=False)
plt.locator_params(axis='x', nbins=3)
plt.grid()

# Plot Volumes
plt.subplot(155, sharey=ax)

y = np.zeros(len(vol))
    
for i, col in enumerate(['VILL', 'VXBW', 'VQUA', 'VCLC', 'VPAR', 'VXWA', 'VXOI', 'VXGA']):
    y += vol[col]
    plt.fill_betweenx(vol.DEPT, y, 0,
                  facecolor=facecolor[col],
                 zorder=-i, lw=0, label=col,
                     hatch=hatch[col])

#plt.legend(loc='upper center', bbox_to_anchor=(-2.95, 1.07), ncol=5)
plt.gca().xaxis.tick_top()
plt.gca().set_xlabel('Volume')    
plt.gca().xaxis.set_label_position('top') 

plt.setp( plt.gca().get_yticklabels(), visible=False)
plt.locator_params(axis='x', nbins=3)
plt.grid(axis='y')
plt.xlim(0,1)

plt.autoscale(enable=True, axis='y', tight=True)
#plt.ylim(1500, 2500)
plt.ylim(plt.ylim()[::-1])

plt.tight_layout()

plt.savefig('well_5_composite_log.pdf')
plt.close()

In [ ]:
## Viewing the results
# You can view the composite logs of both wells using evince

#!evince well_4_composite_log.pdf
#!evince well_5_composite_log.pdf

## Covert logs to LAS format

To make it easier to use these wells, we should convert their logs to LAS format.

# Preliminar time-depth relationship

For a first approximation we can use the VSP times as a time-depth relationship. After we have a well processed seismic we can fine tune it with a proper well tie

In [ ]:
!head well4.Q | cat -n; echo
!head well5.Q | cat -n; echo
!head well7.Q | cat -n; echo

In [ ]:
# Read well 4 data
vsp = pd.read_csv('well4.Q', sep='\s+', names=['DEPT', 'OWT', 'TWT', 'Q'],
                 skiprows=8)
vsp['WELL'] = 'Well 4'

# Read well 5 data
tmp = pd.read_csv('well5.Q', sep='\s+', names=['DEPT', 'OWT', 'TWT', 'Q'],
                 skiprows=8)
tmp['WELL'] = 'Well 5'
vsp = vsp.append(tmp)

# Read well 7 data
tmp = pd.read_csv('well7.Q', sep='\s+', names=['DEPT', 'OWT', 'TWT', 'Q'],
                 skiprows=9)
tmp['WELL'] = 'Well 7'
vsp = vsp.append(tmp)

vsp.TWT /= 1000.
vsp.head()

In [ ]:
vsp['SSTVD'] = vsp.WELL.map(lambda x: metadata[x]['KB']) - vsp.DEPT

In [ ]:
for well, gr in vsp.groupby('WELL'):
    plt.plot(gr.TWT, gr.SSTVD, '.', label=well)
    
plt.ylabel('SSTVD (m)')
plt.xlabel('two-way time (s)')
plt.legend(loc='best')
sns.despine(offset=5)

# Making a RMS velocity curve for Well 5

First we need a shallow layers velocity model, we can build that using first break picks and solving critical refraction equations.

In [ ]:
def rolling_sum(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0))
    return (cumsum[N:] - cumsum[:-N])

def modified_energy_ratio(x,N):
    a = np.append(np.ones(N-1)*np.mean(x[:2]), x)
    a = rolling_sum(a*a,N)

    b = np.append(x, np.ones(N-1)*np.mean(x[:-2]))
    b = rolling_sum(b*b,N)

    return np.power(np.abs(x)*b/(a + 0.0001),3)

In [ ]:
# Let's get the shotpoint on top of Well 5
common_shot = 'line_12_csg_raw.rsf'
data_sample = 'sample_shot.rsf'

# Grab only shotpoint 822
!sfwindow < {common_shot} n3=1 f3=713 > {data_sample}
!sfin {data_sample}

In [ ]:
data_sample = sf.Input(data_sample)

n1 = data_sample.int('n1')
n2 = data_sample.int('n2')
d1 = data_sample.float('d1')
o1 = data_sample.float('o1')

data = np.zeros((n2, n1), dtype=np.float32)
data_sample.read(data)

In [ ]:
picks = []

for trace in data:
    picks.append(np.argmax(modified_energy_ratio(trace, 100)))
    
picks = np.array(picks)
picks = picks[::-1]*d1 + o1

In [ ]:
# Water layer properties
v_water = 1500. #m/s
h_water = metadata['Well 5']['WD'] #water depth
owt_water = h_water/v_water

In [ ]:
t = np.arange(n1) * d1 + o1
x = np.arange(120) * 25 + 262 # Maybe I should read this from the header

perc = np.percentile(data.ravel(), 99)

extent = [x[-1], x[0], t[-1], t[0]]

plt.figure(figsize=(5,6))
plt.imshow(data.clip(-perc, perc).T, aspect='auto', extent=extent, cmap='gray_r')

# Plottin the first break picks
plt.plot(x, picks, color='r', lw=1, label='first break picks')

# Plotting the events
# direct wave
plt.plot(x, x/v_water, 'b--', label='direct wave')
# water bottom reflection
plt.plot(x, 2* np.sqrt(x**2/(4*v_water**2) + (h_water/v_water)**2),
         label='sea floor reflection')

# X_cross for the first layer: https://en.wikipedia.org/wiki/Seismic_refraction
x_cross = 2150 # I picked this from the plot

p1, t01 = np.polyfit(x[x > 2150], picks[x > 2150], deg=1)
v1 = 1/p1

plt.plot(x, t01 + x/v1, label='first layer refraction')


plt.ylabel('t (s)')
plt.xlabel('offset (m)')

plt.title('shotpoint %d' % metadata['Well 5']['EP'])

plt.axis('tight')
plt.ylim(2.5, 0)
sns.despine()

plt.legend(loc='best')

### First layer properties

In [ ]:
ic = np.arcsin(v_water/v1)
h0 = t01*v_water/(2*np.cos(ic))
h0, v1, h0/v1

In [ ]:
# Water layer properties
kb = metadata['Well 5']['KB'] # KB for datum conversions

# RMS velocity for Well 5
rms = pd.DataFrame({'DEPT' : [kb, h_water + kb, h_water + kb + h0],
                    'SSTVD' : [0, -h_water, -h_water - h0],
                    'OWT' : [0, owt_water, owt_water + h0/v1]})

rms['TWT'] = rms.OWT * 2

rms = rms.append(vsp.query('WELL == "Well 5"')[['DEPT', 'SSTVD', 'OWT', 'TWT']])
rms.sort_values('DEPT', inplace=True)
rms.reset_index(inplace=True, drop=True)

In [ ]:
rms.head()

In [ ]:
rms['VINT'] = rms.DEPT.diff()/rms.OWT.diff()
rms.VINT = rms.VINT.fillna(1500)

while(np.sum(~rms.VINT.between(1500, 8000)) > 0):
    rms = rms[rms.VINT.between(1500, 8000)]
    rms['VINT'] = rms.DEPT.diff()/rms.OWT.diff()
    rms.VINT = rms.VINT.fillna(1500)

In [ ]:
rms['VRMS'] = np.sqrt(np.cumsum(rms.VINT**2 * rms.OWT.diff())/np.cumsum(rms.OWT.diff()))
rms.VRMS = rms.VRMS.fillna(1500)

plt.figure(figsize=(3,6))
plt.plot(rms.VRMS, rms.SSTVD, color='r', label='V$_{RMS}$')
plt.step(rms.VINT, rms.SSTVD, color='g', where='post', label='V$_{int}$')
plt.axhline(-metadata['Well 5']['WD'], ls='--', label='sea floor')
plt.axhline(-h_water - h0, color='k', ls='--', label='first layer')

plt.xlabel('velocity (m/s)')
plt.ylabel('SSTVD (m)')

plt.title('Well 5 RMS velocity profile')

plt.legend(loc='best')
sns.despine(offset=5)

In [ ]:
rms.to_csv('well_5_rms_profile.txt', index=False)